In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re


In [2]:
data = pd.read_csv('./data/Consumidor_Venta_Producto_UPC_Recom_2018_2020.csv')
english_cols = {'FACTURA_ID': 'INVOICE_ID', 'FACTURA_POSICION_ID': 'INVOICE_POSITION_ID', 'CUSTOMER_ID': 'CUSTOMER_ID',
                'FACTURA_CLASE_DOCUMENTO_ID': 'INVOICE_DOCUMENT_CLASS_ID', 'ANO_MES_FACTURA': 'INVOICE_MONTH_YEAR',
                'ANO_FACTURA': 'ORDER_YEAR', 'MES_FACTURA': 'ORDER_MONTH', 'FECHA_FACTURA': 'INVOICE_DATE',
                'IMP_VENTA_NETO_EUR': 'NET_SALES_EUR', 'CANAL_VENTA_ID': 'SALES_CHANNEL_ID',
                'CANAL_VENTA_DESC': 'SALES_CHANNEL_DESC', 'TEMPORADA_COMERCIAL_ID': 'SEASON_ID',
                'TEMPORADA_COMERCIAL_DESC': 'SEASON_DESC', 'PRODUCTO_ID': 'PRODUCT_ID', 'TALLA': 'SIZE',
                'MATERIAL_ID': 'MATERIAL_ID', 'NUMERO_DEUDOR_PAIS_ID': 'SALE_COUNTRY_ID',
                'NUMERO_DEUDOR_PAIS_DESC': 'SALE_COUNTRY_DESC', 'VENTA_DEVOLUCION': 'SALES_FILTER',
                'JERARQUIA_PROD_ID': 'PRODUCT_HIERARCHY_ID', 'GRUPO_ARTICULO_PRODUCTO_ID': 'PRODUCT_GROUP_ID',
                'GRUPO_ARTICULO': 'PRODUCT_GROUP', 'CONCEPTO': 'CONCEPT', 'LINEA': 'LINE',
                'GENERO_PRODUCTO': 'PRODUCT_GENDER', 'CATEGORIA': 'PRODUCT_CATEGORY', 'TIPOLOGIA': 'PRODUCT_TYPE',
                'COLOR': 'COLOR_INTERN', 'CONSUMER_COLOR': 'COLOR_EXTERN', 'CREMALLERA': 'ZIPPER', 'CORDONES': 'LACES',
                'OUTSOLE_SUELA_TIPO': 'SOLE_TYPE', 'OUTSOLE_SUELA_SUBTIPO': 'SOLE_SUBTYPE',
                'PLANTILLA_EXTRAIBLE': 'REMOVABLE_SOLE', 'CONTACTO_SN': 'PHYSICAL_CONTACT', 'EDAD_SN': 'AGE_AVAILABLE',
                'GENERO_CONTACTO': 'GENDER_AVAILABLE', 'EDAD_COMPRA': 'AGE_AT_PURCHASE',
                'EDAD_RANGO_COMPRA': 'AGE_RANGE',
                'PAIS_CONTACTO': 'COUNTRY_CONTACT_ID', 'PAIS_CONTACTO_DESC': 'COUNTRY_CONTACT_DESC',
                'CIUDAD_CONTACTO': 'CITY_CONTACT', 'IDIOMA_CONTACTO': 'LANGUAGE_CONTACT'}


In [3]:
dropcols = ['INVOICE_MONTH_YEAR', 'ORDER_YEAR','ORDER_MONTH','SALES_CHANNEL_ID',\
            'SALES_CHANNEL_DESC','AGE_AVAILABLE','MATERIAL_ID','COUNTRY_CONTACT_DESC',\
            'SALE_COUNTRY_DESC','SALES_FILTER','SEASON_ID','GENDER_AVAILABLE']

In [4]:
# gets most common country for the code
def country_dict(df, id_field,desc_field):
    country_dict = {x[id_field]:x[desc_field] for x in df.groupby(id_field)[desc_field].apply(lambda x: x.mode()).to_frame().reset_index().to_dict('index').values()}
    return country_dict

In [5]:
country_lookup = country_dict(data, 'PAIS_CONTACTO','PAIS_CONTACTO_DESC')
country_lookup.update(country_dict(data, 'NUMERO_DEUDOR_PAIS_ID','NUMERO_DEUDOR_PAIS_DESC'))
country_lookup



{'AD': 'ANDORRA',
 'AE': 'EMIRATOS ÁRABES UNIDOS',
 'AF': 'AFGANISTÁN',
 'AG': 'ANTIGUA Y BARBUDA',
 'AL': 'ALBANIA',
 'AR': 'ARGENTINA',
 'AT': 'AUSTRIA',
 'AU': 'AUSTRALIA',
 'AZ': 'AZERBAYÁN',
 'BA': 'BOSNIA-HERZEGOVINA',
 'BE': 'BÉLGICA',
 'BG': 'BULGARIA',
 'BR': 'BRAZIL',
 'BY': 'BIELORRUSIA',
 'CA': 'CANADA',
 'CH': 'SWITZERLAND',
 'CL': 'CHILE',
 'CN': 'CHINA',
 'CO': 'COLOMBIA',
 'CR': 'COSTA RICA',
 'CU': 'CUBA',
 'CY': 'CHIPRE',
 'CZ': 'REPÚBLICA CHECA',
 'DE': 'GERMANY',
 'DK': 'DENMARK',
 'EC': 'ECUADOR',
 'EE': 'ESTONIA',
 'EG': 'EGIPTO',
 'ES': 'SPAIN',
 'FI': 'FINLANDIA',
 'FR': 'FRANCE',
 'GB': 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND',
 'GE': 'GEORGIA',
 'GP': 'GUADALUPE',
 'GR': 'GREECE',
 'HK': 'HONG KONG',
 'HR': 'CROACIA',
 'HU': 'HUNGRÍA',
 'IE': 'IRELAND',
 'IL': 'ISRAEL',
 'IN': 'INDIA',
 'IS': 'ISLANDIA',
 'IT': 'ITALY',
 'JP': 'JAPAN',
 'KR': 'KOREA REPUBLIC OF',
 'KZ': 'KAZAJSTÁN',
 'LA': 'LAOS',
 'LI': 'LIECHTENSTEIN',
 'LT': 'LITUANIA',
 'LU':

In [116]:
def preprocess(d):
    d_copy = d.rename(columns=english_cols)
    d_copy['REMOVABLE_SOLE'] = d_copy['REMOVABLE_SOLE'].apply(lambda x: True if x == 'Extraible' else False)
    
    invoice_id_dict = {'ZTON':'Sale','ZDVN':'Return','ZDAN':'Cancelled Return'}
    d_copy['INVOICE_DOCUMENT_CLASS_ID'] = d_copy['INVOICE_DOCUMENT_CLASS_ID'].apply(lambda x: invoice_id_dict[x])
    
    d_copy['INVOICE_DATE'] = pd.to_datetime(d_copy.INVOICE_DATE)
    
    d_copy.AGE_AT_PURCHASE = d_copy.AGE_AT_PURCHASE.replace(0, np.nan)
    
    d_copy.ZIPPER = d_copy.ZIPPER.apply(lambda x: True if x in ('SI','YES') else False)
    d_copy.LACES = d_copy.LACES.apply(lambda x: True if x in ('With laces', 'Con cordones') else False)
    
    product_group_dict = {'Zapatos Adulto':'Adult Shoes',
                          'Bolsos':'Bag',
                          'Bolsos Cartujano':'Bag',
                          'Ropa':'Clothings',
                          'Complementos':'Complements'
                         }
    d_copy.PRODUCT_GROUP = d_copy.PRODUCT_GROUP.apply(lambda x: product_group_dict[x] if x in product_group_dict else x)
    
    color_dict = {'Мульти ассорти':'Multi - Assorted',
                  'красный': 'Red',
                  'розовый' : 'Pink',
                  'желтый' : 'Yellow'
                 }
    d_copy.COLOR_INTERN = d_copy.COLOR_INTERN.apply(lambda x: color_dict[x] if x in color_dict else x)
    
    d_copy.PHYSICAL_CONTACT = d_copy.PHYSICAL_CONTACT.apply(lambda x: True if x == 'S' else False)
    
    d_copy.SEASON_DESC = d_copy.SEASON_DESC.apply(lambda x: "".join(re.split('^\d{2}|\W+', x)))
    
    d_copy.SOLE_SUBTYPE = d_copy.SOLE_SUBTYPE.apply(lambda x: np.nan if pd.isnull(x) else "".join(re.split('\(.*\)', x)))
    
    
    return d_copy



In [120]:
cleandata = preprocess(data)
cleandata = cleandata.drop(columns=dropcols)



In [127]:
item_fields = ['PRODUCT_ID','PRODUCT_GROUP','CONCEPT','LINE',\
               'PRODUCT_GENDER','PRODUCT_CATEGORY','PRODUCT_TYPE',\
               'LACES','ZIPPER','SOLE_TYPE','SOLE_SUBTYPE','REMOVABLE_SOLE','SEASON_DESC',\
              'COLOR_INTERN']

item_bools = ['LACES','ZIPPER','REMOVABLE_SOLE']

item_categoricals = ['PRODUCT_GROUP','CONCEPT','LINE','PRODUCT_GENDER',\
                     'PRODUCT_CATEGORY','PRODUCT_TYPE','SOLE_TYPE',\
                     'SOLE_SUBTYPE','SEASON_DESC','COLOR_INTERN'
                    ]


In [153]:
cleandata

,INVOICE_ID,INVOICE_POSITION_ID,CUSTOMER_ID,INVOICE_DOCUMENT_CLASS_ID,INVOICE_DATE,NET_SALES_EUR,SEASON_DESC,PRODUCT_ID,SIZE,SALE_COUNTRY_ID,...,LACES,SOLE_TYPE,SOLE_SUBTYPE,REMOVABLE_SOLE,PHYSICAL_CONTACT,AGE_AT_PURCHASE,AGE_RANGE,COUNTRY_CONTACT_ID,CITY_CONTACT,LANGUAGE_CONTACT
0,2945652414,1,rFT69H3UGh2ARZpBvvotibgvZ6EXIucsbnz1Zxedtwg=,Sale,2019-11-29,142.86,FW2019,K400208-001,40,DE,...,False,HEEL,HIGH,False,True,NaN,NS,DE,Berlin,DE
1,2911680032,2,DZybTgMUb57l6e9/6VwWNFBT6FCX7iup9nv9FzNmwmg=,Sale,2019-11-29,85.74,FW2019,36587-052,41,RO,...,True,FLAT,FLAT,False,True,NaN,NS,RO,Buzau,EN
2,2911679783,1,7E4cndT60vWzB14fXfrimhksQFzOLMmgti4TStQcUF4=,Sale,2019-11-29,119.84,FW2019,K200734-007,38,ES,...,True,FLAT,FLAT,False,True,NaN,NS,ES,Salteras,ES
3,2943639042,1,L6ZbTVDy9xVx8Y0K/oRK6QVfw+OMpi43PAQ1ehjMcHc=,Sale,2019-11-29,79.94,FW2019,K300143-003,43,GB,...,True,FLAT,FLAT,True,True,56.0,50-59,GB,LONDON,EN
4,2911679122,2,+33BsbHHCTxIe9/1KGjYFa9Y4CCMwHnXehsJtefOWLo=,Sale,2019-11-29,164.25,FW2019,K300287-002,43,ES,...,True,FLAT,FLAT,True,True,NaN,NS,ES,oviedo,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327378,2043647241,4,SlMvRc0xQJ4hXQoi/ZAz88/Nv5z4cn1n59sIzJEL7jw=,Sale,2020-12-31,103.56,FW2020,K300281-002,43,GB,...,True,FLAT,FLAT,False,True,63.0,60-69,GB,Northampton,EN
1327379,2011713438,1,9al4cCmXbaTW87N0pioI/sY+HDyCD6xFF81VX89zUJY=,Sale,2020-12-31,227.27,FW2020,K400384-006,41,LT,...,False,WEDGE,HIGH,False,True,29.0,20-29,LT,Vilnius,EN
1327380,2043647282,1,tQXERK43eGtEMw/q4/GFVuT1ZVl796Wkw3+/6llN+1o=,Sale,2020-12-31,161.91,FW2020,K300005-017,43,GB,...,False,FLAT,FLAT,False,True,47.0,40-49,GB,London,EN
1327381,2011713256,1,RR2UV985hr1GXfdVp0wsnAeq53fdg2npTmcfk50o6qo=,Sale,2020-12-31,151.86,FW2020,K300285-010,44,FI,...,True,FLAT,FLAT,True,True,58.0,50-59,FI,Helsinki [Helsingfors],EN


In [152]:
items_attributes = cleandata[item_fields]
items_attributes


,PRODUCT_ID,PRODUCT_GROUP,CONCEPT,LINE,PRODUCT_GENDER,PRODUCT_CATEGORY,PRODUCT_TYPE,LACES,ZIPPER,SOLE_TYPE,SOLE_SUBTYPE,REMOVABLE_SOLE,SEASON_DESC,COLOR_INTERN
0,K400208-001,Adult Shoes,Kara,Kara,WOMEN,Women Ankle Boot,Zip Bootie,False,True,HEEL,HIGH,False,FW2019,Black
1,36587-052,Adult Shoes,Mil,1913,MEN,Men Ankle Boot,Desert Boot,True,False,FLAT,FLAT,False,FW2019,Black
2,K200734-007,Adult Shoes,Tyra,Tyra,WOMEN,Women Shoe,Blucher,True,False,FLAT,FLAT,False,FW2019,Medium Gray
3,K300143-003,Adult Shoes,Andratx,Andratx,MEN,Men Sneaker Boot,Sneaker bootie,True,True,FLAT,FLAT,True,FW2019,Black
4,K300287-002,Adult Shoes,Peu,Peu Pista GM,MEN,Men Mid Boot,Medium Zip / Medium Elastic,True,True,FLAT,FLAT,True,FW2019,Medium Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327378,K300281-002,Adult Shoes,Formiga,Formiga,MEN,Men Ankle Boot,Desert Boot,True,False,FLAT,FLAT,False,FW2020,Medium Brown
1327379,K400384-006,Adult Shoes,Kaah,Kaah,WOMEN,Women Mid boot,Medium Boot,False,True,WEDGE,HIGH,False,FW2020,Black
1327380,K300005-017,Adult Shoes,Beetle,Beetle,MEN,Men Ankle Boot,Basket bootie,False,False,FLAT,FLAT,False,FW2020,Black
1327381,K300285-010,Adult Shoes,Peu,Peu Pista GM,MEN,Men Ankle Boot,Basket bootie,True,False,FLAT,FLAT,True,FW2020,Dark Brown


In [144]:

onehot_categoricals = pd.get_dummies(items_attributes[item_categoricals])


In [146]:
item_mtx_df = pd.concat([items_attributes.PRODUCT_ID, items_attributes[item_bools]*1, onehot_categoricals], axis=1)


In [147]:
item_mtx_df

,PRODUCT_ID,LACES,ZIPPER,REMOVABLE_SOLE,PRODUCT_GROUP_Adult Shoes,PRODUCT_GROUP_Bag,PRODUCT_GROUP_Clothings,PRODUCT_GROUP_Complements,PRODUCT_GROUP_Kid's Shoes,PRODUCT_GROUP_Kids,...,COLOR_INTERN_Multi - Assorted,COLOR_INTERN_Navy,COLOR_INTERN_No Color,COLOR_INTERN_Nude,COLOR_INTERN_Pink,COLOR_INTERN_Red,COLOR_INTERN_Rust/Copper,COLOR_INTERN_Turquiose Aqua,COLOR_INTERN_White Natural,COLOR_INTERN_Yellow
0,K400208-001,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36587-052,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,K200734-007,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,K300143-003,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,K300287-002,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327378,K300281-002,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327379,K400384-006,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327380,K300005-017,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1327381,K300285-010,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
item_mtx_np = np.asmatrix(item_mtx_df)

In [158]:
item_mtx_np.shape

(1327383, 942)